# Assignment 1: Chinese character "detection"
LT2326, Autumn 2021

Name: Max Boholm (gusbohom)

# Introduction

This notebook cotains the code for *Assignment 1: Chinese character "detection"* for the course *Machine learning for statistical NLP: Advanced* (course code LT2326), Autumn 2021. The notebook is organized into the folowing parts:

*    Meta variables (the term *hyperparameter* is here reserved for decisions on the models), which define ... the loacation (path) of the data, ... 
*    Data preparation
*    Definition and training of two models
*    Testing and evaluation
*    ...




# Libraries

In [2]:
import glob
import json
import random
import torch
from PIL import Image
import numpy as np

# Meta variables

In [3]:
#path = "../../scratch/lt2326-h21/a1/"
path = "../develop_util/"
images_dir = path + "images"      # the directory of images to be data
meta_ctw   = path + "info.json"   # the (path to) the general meta file of CTW 
meta_train = path + "train.jsonl" # the (path to) the file containing the annotations of CTW training data

train_proportion = 0.7 # the proportion of training data; proportion of test data will be the complement of this number

restriction = None # set to an integer, if subsample of the data is to be used in e.g. the development phase

#device = torch.device('cuda:0')
device = torch.device('cpu')


## Hyperparameters

In [ ]:
batch_size = ...
lr = ... 
stride = ...
window = ...
... 


## Part 1: data preparation

In [5]:
def only_train(file_dir = images_dir, meta_file = meta_ctw):
    """Tests whether image files in a directory is part of the training set of the CTW data set, 
       as defined by the json metafile for CTW. Returns a list of filenames such that they are both
       in the specified directory and the CTW training set.
    """
    meta = json.load(open(meta_file,"r"))
    
    train_files_CTW = [entry["file_name"] for entry in meta["train"]]
        
    files_to_keep = []
    
    for file in glob.glob(file_dir+"*.jpg"):
        if file in train_files_CTW:
            files_to_keep.append(file)
        
    return files_to_keep

def CTW_mapper(files, meta = meta_train):
    """ Identifies annotations for files from the training set of the CTW dataset. 
        Returns pyhoton dictionary that maps filenames (keys) with annotations (values), 
        which like in the original format is a list of lists of json elements / python dictinaries. 
    """
    mapping = {}
   
    with open(meta, "r") as f:
        
        annotations_data = f.readlines()

        for file in files:
            for annotation in annotations_data:
                if annotation["image_id"] == file[:-4]:
                    mapping[file] = annotations_data["annotations"]
                    break
    
    return mapping
                    
def img2vector(file, directory=images_dir, device=device):
    """ Takes a filename of an image in a directory and returns a ...
    """
    
    img = Image.open(directory+file)
    img_vec = torch.LongTensor(np.array(img), device=device) # do I really need this nesting?
    
    #flatten?
    #padding?
    
    return img_vec

def coordinates2vector(file, mapping, device=device, height = 2048, width = 2048): 
    """ Builds a vector of 0s and 1s based on the coordinates ...
    """
    
    char_matrix = torch.zeros(height, width, device=device)
    
    char_areas = []
    for block in mapping[file]:
        for character in block:
            if character["is_chinese"] == True:
                xmin = character["adjusted_bbox"][0]
                ymin = character["adjusted_bbox"][1]
                w    = character["adjusted_bbox"][2]
                h    = character["adjusted_bbox"][3]
                char_areas.append((xmin, ymin, w, h))
    
    for xmin, ymin, w, h in char_areas:
        
        r1 = height - ymin - h
        r2 = r1 + h
        c1 = width - xmin
        c2 = c1 + w
        
        char_matrix[r1:r2 , c1:c2] = 1
    
    #flatten?
    
    return char_matrix
        
def data_builder(files, directory, mapping, restriction = restriction):
    """ Compiles the data set for use. Returns a list of dictionaties, specifying:
        -  the filename; key: "file"
        -  the training data (vector); key: "img_vector"
        -  the labels (a vector of 0s and 1s indicating boxes of characters in images); key: "label" 
    """
    
    data = []
    
    for file in files:
        instance = {}
        instance["file"] = file
        instance["img_vector"] = img2vector(file)
        instance["label"] = coordinates2vector(file, mapping)
        data.append(d) 
    
    if restriction != None:
        random.shuffle(data)
        data = data[:restriction]
        
    return data
    
files = only_train()
mapping = CTW_mapper(files, path)
my_data = data_builder(files, images_dir, mapping)

IsADirectoryError: [Errno 21] Is a directory: '../develop_util/'

In [ ]:
def split():
    pass

train_data, test_data = split(my_data)

In [ ]:
# we need padding!



#to be used in training below ... 
def data_loader(data, batch):
    ...
    yield ...

In [ ]:
import torch
t = torch.zeros(10, 10)
t[1:3, 5:7]=1 # rows, columns
print(t.shape)
t


In [ ]:
from PIL import Image
import torch
import numpy as np
img = Image.open("../develop_util/images/0000172.jpg")

img_np=np.array(img)
img_vec=torch.LongTensor(img_np)
#print(img)
img_vec

## Part 2: the models

### Model 1: ...

In [ ]:
m1_name = ""

#### Model definition

#### Training and saving model

### Model 2: ...

In [ ]:
m2_name = ""

#### Model definition

#### Training and saving model

## Part 3: testing and evaluation